# Project: Question-Answering on Private Documents (RAG)


In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [2]:
pip install -q chromadb

Note: you may need to restart the kernel to use updated packages.


In [9]:
# Function to load a PDF file using LangChain's PyPDFLoader
def load_document(file_path):
    from langchain.document_loaders import PyPDFLoader
    loader = PyPDFLoader(file_path)
    return loader.load()

# Function to chunk the document into smaller pieces
def chunk_data(data, chunk_size=256, chunk_overlap=20):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.split_documents(data)


In [14]:
def ask_and_get_answer(vector_store, q, k=3):
    from langchain.chains import RetrievalQA
    from langchain_openai import ChatOpenAI

    llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1)

    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': k})

    chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
    
    answer = chain.invoke(q)
    return answer
    

In [15]:
def create_embeddings_chroma(chunks, persist_directory='./chroma_db'):
    from langchain.vectorstores import Chroma
    from langchain_openai import OpenAIEmbeddings

    # Instantiate an embedding model from OpenAI (smaller version for efficiency)
    embeddings = OpenAIEmbeddings(model='text-embedding-3-small', dimensions=1536)  

    # Create a Chroma vector store using the provided text chunks and embedding model, 
    # configuring it to save data to the specified directory 
    vector_store = Chroma.from_documents(chunks, embeddings, persist_directory=persist_directory) 

    return vector_store  # Return the created vector store


In [16]:
def load_embeddings_chroma(persist_directory='./chroma_db'):
    from langchain.vectorstores import Chroma
    from langchain_openai import OpenAIEmbeddings

    # Instantiate the same embedding model used during creation
    embeddings = OpenAIEmbeddings(model='text-embedding-3-small', dimensions=1536) 

    # Load a Chroma vector store from the specified directory, using the provided embedding function
    vector_store = Chroma(persist_directory=persist_directory, embedding_function=embeddings) 

    return vector_store  # Return the loaded vector store


In [17]:
# Loading the pdf document into LangChain 
data = load_document('psychologyMoney.pdf')

# Splitting the document into chunks
chunks = chunk_data(data, chunk_size=256)

# Creating a Chroma vector store using the provided text chunks and embedding model (default is text-embedding-3-small)
vector_store = create_embeddings_chroma(chunks)

In [20]:
# Asking questions
q = 'what is Milanković’s theory?'
answer = ask_and_get_answer(vector_store, q)
print(answer)

{'query': 'what is Milanković’s theory?', 'result': "Milanković’s theory proposes that variations in Earth's orbit and tilt cause changes in the intensity of sunlight received at different latitudes and times of the year, leading to changes in climate and the onset of ice ages. Initially, it was believed that these changes caused extremely cold winters, but later research by Wladimir Köppen suggested that moderately cool summers were actually the key factor in the freezing process."}


In [21]:
print(answer['result'])

Milanković’s theory proposes that variations in Earth's orbit and tilt cause changes in the intensity of sunlight received at different latitudes and times of the year, leading to changes in climate and the onset of ice ages. Initially, it was believed that these changes caused extremely cold winters, but later research by Wladimir Köppen suggested that moderately cool summers were actually the key factor in the freezing process.


In [33]:
# Load a Chroma vector store from the specified directory (default ./chroma_db) 
db = load_embeddings_chroma()
q = 'Derek Thompson of The Atlantic says what?'
answer = ask_and_get_answer(vector_store, q)
print(answer)

{'query': 'Derek Thompson of The Atlantic says what?', 'result': 'This quote from Derek Thompson of The Atlantic highlights how portable devices have made the modern factory location-independent, as work can be done anywhere, turning the entire day into a productive period.'}


## Adding Memory (Chat History)

In [38]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ChatMessageHistory, ConversationBufferMemory
from langchain.vectorstores import Chroma

# Loading vector store (assumes it's already created or loaded)
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 5})

# Create memory object for the chat history
history = ChatMessageHistory()
memory = ConversationBufferMemory(
    chat_memory=history, 
    memory_key="chat_history",  # ✅ Specify memory key
    return_messages=True
)

# Instantiate the LLM
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

# Create the ConversationalRetrievalChain
crc = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    chain_type='stuff',
    verbose=False
)

# function that handles chat_history properly
def ask_question_v1(q, chain):
    # Pass both question and empty chat_history for first question
    result = chain.invoke({
        'question': q,
        'chat_history': []  # Empty for first question, will be managed by memory
    })
    return result

# Alternative function that relies entirely on memory
def ask_question_v2(q, chain):
    # Since we're using memory, we can just pass the question
    # The chain will handle chat_history through the memory component
    result = chain.invoke({'question': q})
    return result

# More robust function with error handling
def ask_question_v3(q, chain):
    try:
        # Try with just the question first (relies on memory)
        result = chain.invoke({'question': q})
        return result
    except ValueError as e:
        if "chat_history" in str(e):
            # If chat_history is required, provide it
            result = chain.invoke({
                'question': q,
                'chat_history': []
            })
            return result
        else:
            raise e

# Load and process the document
data = load_document('psychologyMoney.pdf')
chunks = chunk_data(data, chunk_size=256)
vector_store = create_embeddings_chroma(chunks)

# Test the question
q = 'Why do people often misjudge the impact of compounding over time?'

try:
    result = ask_question_v1(q, crc)  # Use solution 1
    print("Answer:", result['answer'])
    print("Source documents:", len(result.get('source_documents', [])))
except Exception as e:
    print(f"Error: {e}")

# For subsequent questions, the memory will maintain the chat history automatically
q2 = "Can you give me a specific example of this misjudgment?"
try:
    result2 = ask_question_v1(q2, crc)
    print("\nSecond Answer:", result2['answer'])
except Exception as e:
    print(f"Error on second question: {e}")

Answer: People often misjudge the impact of compounding over time because they are accustomed to how quickly things can grow. This familiarity with rapid growth can lead individuals to overlook the power of compounding, causing them to focus on solving problems through other means instead.
Source documents: 0

Second Answer: One specific example of why people often misjudge the impact of compounding over time is the concept of exponential growth. When investments or savings grow exponentially over time due to compounding, the growth rate accelerates as the base amount increases. This means that even small contributions or returns can lead to significant wealth accumulation over long periods. However, because exponential growth is not intuitive for many people, they may underestimate the long-term impact of compounding and delay starting to save or invest.


## Using a Custom Prompt

In [45]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain

# Setup
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 5})
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

# Custom prompt template
custom_template = """You are a financial psychology expert. Answer the question using the context provided.

Context: {context}
Question: {question}

Provide a clear, detailed answer with practical examples:"""

custom_prompt = PromptTemplate(
    template=custom_template,
    input_variables=["context", "question"]
)

# Create chain with custom prompt
crc = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": custom_prompt}
)

# Ask questions
def ask_question(question, chain):
    result = chain.invoke({'question': question, 'chat_history': []})
    return result['answer']

# Usage
data = load_document('psychologyMoney.pdf')
chunks = chunk_data(data, chunk_size=256)
vector_store = create_embeddings_chroma(chunks)

q = 'Why do people often misjudge the impact of compounding over time?'
answer = ask_question(q, crc)
print(answer)

People often misjudge the impact of compounding over time because they are accustomed to how quickly things can grow in other areas of their lives. In today's fast-paced society, we are used to instant gratification and immediate results. This mindset can lead people to overlook the power of compounding, which involves gradual growth over time.

For example, when it comes to investing, individuals may be more inclined to seek out quick gains or high-risk investments that promise immediate returns. They may overlook the potential of compounding interest, which can significantly increase their wealth over the long term. Similarly, in personal finance, people may prioritize paying off debt quickly rather than taking advantage of the benefits of compounding by investing or saving for the future.

Additionally, the concept of compounding may not be intuitive to everyone, leading them to underestimate its impact. It can be challenging for individuals to grasp the idea that small, consistent 